***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# \**Zusatz Vorlesung 3: Konstituententests und Grammatiken*


In [1]:
import nltk



## Konstituententests: Beispiel `ADJP` (Eliminierungstests)

Ergebnis Eliminierungstest: `der sehr große Hund` > `*der sehr Hund`: ungrammatisch

> Eine unter Erhalt der Grammatikalität eines Satzes weglassbare Wortfolge ist Konstituente

In [2]:
# Grammatik als Ergebnis der folgenden Konstituententests (Eliminierungstests):
grammar = nltk.CFG.fromstring("""
    NP -> Det ADJP N
    ADJP -> Prt Adj
    ADJP -> Adj
    Det -> 'der'
    N -> 'Hund'
    Adj -> 'große' | 'kleine'
    Prt -> 'sehr'
""")

parser = nltk.ShiftReduceParser(grammar,trace=2)

In [3]:
sent = 'der sehr große Hund'.split()
for tree in parser.parse(sent):
    print('\n',tree,'\n')
    tree.pretty_print(unicodelines=True)

Parsing 'der sehr große Hund'
    [ * der sehr große Hund]
  S [ 'der' * sehr große Hund]
  R [ Det * sehr große Hund]
  S [ Det 'sehr' * große Hund]
  R [ Det Prt * große Hund]
  S [ Det Prt 'große' * Hund]
  R [ Det Prt Adj * Hund]
  R [ Det ADJP * Hund]
  S [ Det ADJP 'Hund' * ]
  R [ Det ADJP N * ]
  R [ NP * ]

 (NP (Det der) (ADJP (Prt sehr) (Adj große)) (N Hund)) 

     NP                 
 ┌───┴────┬──────────┐   
 │       ADJP        │  
 │   ┌────┴─────┐    │   
Det Prt        Adj   N  
 │   │          │    │   
der sehr      große Hund



In [4]:
# 'sehr große' ist eliminierbar (unter Erhalt der Grammatikalität) 
#    > Konstituente (ADJP):
sent = 'der Hund'.split()
for tree in parser.parse(sent):
    print('\n',tree,'\n')
    tree.pretty_print(unicodelines=True)

Parsing 'der Hund'
    [ * der Hund]
  S [ 'der' * Hund]
  R [ Det * Hund]
  S [ Det 'Hund' * ]
  R [ Det N * ]


In [5]:
#Eliminierung Prt 'sehr' unter Erhalt der Grammatikalität 
#    > Adj ist Kopf der Konstituente = ADJP:
sent = 'der große Hund'.split()
for tree in parser.parse(sent):
    print('\n',tree,'\n')
    tree.pretty_print(unicodelines=True)

Parsing 'der große Hund'
    [ * der große Hund]
  S [ 'der' * große Hund]
  R [ Det * große Hund]
  S [ Det 'große' * Hund]
  R [ Det Adj * Hund]
  R [ Det ADJP * Hund]
  S [ Det ADJP 'Hund' * ]
  R [ Det ADJP N * ]
  R [ NP * ]

 (NP (Det der) (ADJP (Adj große)) (N Hund)) 

      NP      
 ┌────┼────┐   
 │   ADJP  │  
 │    │    │   
Det  Adj   N  
 │    │    │   
der große Hund



In [6]:
#Eliminierung Adj 'große' ohne gleichzeitig Elim. von 'sehr' nicht möglich
#    > ungrammatische Struktur (~ Grammatik findet keine Ableitung)
#    > Ergebnis Eliminierungstest: 'große' ist keine Konstituente!
sent = 'der sehr Hund'.split()
for tree in parser.parse(sent):
    print('\n',tree,'\n')
    tree.pretty_print(unicodelines=True)

Parsing 'der sehr Hund'
    [ * der sehr Hund]
  S [ 'der' * sehr Hund]
  R [ Det * sehr Hund]
  S [ Det 'sehr' * Hund]
  R [ Det Prt * Hund]
  S [ Det Prt 'Hund' * ]
  R [ Det Prt N * ]


#### aber Ergebnis Substitutionstest: Adj ist Konstituente

> Eine unter Erhalt der Grammatikalität des Satzes durch eine andere Wortfolge ersetzbare Wortfolge ist Konstituente

#### Merke: Eliminierungstest findet nur optionale Konstituenten (s.u. Dependenzanalyse) 
- also Modifikatoren wie das Attribut *sehr große* oder die optionale Erweiterung des Adjektivs um den Gradpartikel *sehr*

In [7]:
#Substitution des Adj möglich
sent = 'der sehr kleine Hund'.split()
for tree in parser.parse(sent):
    print('\n',tree,'\n')
    tree.pretty_print(unicodelines=True)

Parsing 'der sehr kleine Hund'
    [ * der sehr kleine Hund]
  S [ 'der' * sehr kleine Hund]
  R [ Det * sehr kleine Hund]
  S [ Det 'sehr' * kleine Hund]
  R [ Det Prt * kleine Hund]
  S [ Det Prt 'kleine' * Hund]
  R [ Det Prt Adj * Hund]
  R [ Det ADJP * Hund]
  S [ Det ADJP 'Hund' * ]
  R [ Det ADJP N * ]
  R [ NP * ]

 (NP (Det der) (ADJP (Prt sehr) (Adj kleine)) (N Hund)) 

     NP                  
 ┌───┴────┬───────────┐   
 │       ADJP         │  
 │   ┌────┴─────┐     │   
Det Prt        Adj    N  
 │   │          │     │   
der sehr      kleine Hund



--- 

### Vergleich mit Dependenzstruktur:

In [8]:
# Grammatik als Ergebnis der folgenden Eliminierungstests:
grammar = nltk.DependencyGrammar.fromstring("""
    'Hund' -> 'der' | 'große'
    'große' -> 'sehr'
    """)

parser = nltk.ProjectiveDependencyParser(grammar)

In [9]:
sent = 'der sehr große Hund'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

    Hund      
 ┌───┴─────┐   
 │       große
 │         │   
der       sehr



In [10]:
# 'sehr große' ist eliminierbar 
#    > optionale Einheit (Attribut) 
sent = 'der Hund'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

Hund
 │   
der 



In [11]:
#Prt 'sehr' ist eliminierbar
#    >'sehr' ist optionale Einheit (Dependent von Adjektiv)
sent = 'der große Hund'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

    Hund      
 ┌───┴─────┐   
der      große



In [12]:
#Eliminierung Adj nicht möglich 
#    > 'sehr' kann als Dependent nicht ohne seinen Kopf ('große') auftreten
sent = 'der sehr Hund'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)